In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyproj
import geopandas as gpd
from shapely.geometry import Point

In [2]:
train = pd.read_csv("30_Training Dataset_V2/training_data.csv")
train.head()

,ID,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,...,建物面積,車位面積,車位個數,橫坐標,縱坐標,備註,主建物面積,陽台面積,附屬建物面積,單價
0,TR-1,台北市,大安區,敦化南路二段,-0.256716,None,11,11,住家用,鋼筋混凝土造,...,-0.174154,-0.819326,0.0,305266,2768378,NaN,0.393926,0.183700,-0.438452,4.627714
1,TR-2,台北市,萬華區,水源路,0.100134,None,7,12,住家用,鋼筋混凝土造,...,0.314204,-0.819326,0.0,300677,2767990,NaN,-0.316131,0.608577,-0.438452,1.887258
2,TR-3,高雄市,鳳山區,北忠街,0.181921,None,10,15,集合住宅,其他,...,0.423366,0.161624,1.0,184815,2504666,NaN,-0.098871,-0.360620,1.525881,1.489072
3,TR-4,新北市,新莊區,福前街,0.085594,None,9,14,集合住宅,鋼筋混凝土造,...,0.164249,0.524653,1.0,296653,2772355,NaN,-0.071147,0.315088,0.231984,2.051217
4,TR-5,新北市,板橋區,文化路一段,-0.938116,None,41,43,住家用,鋼骨造,...,0.985839,0.532377,1.0,297377,2768472,NaN,0.791954,1.719400,-0.438452,3.269198


# 計算最近設施實際距離

In [3]:
facility_data = pd.read_csv("landmark.csv")
facility_data.head(5)

,縣市代號,縣市,大類別,小類別代號,小類別名稱,地標名稱,緯度,經度
0,A,臺北市,交通運輸,11,火車站,臺北市臺鐵南港車站,25.053541,121.607264
1,A,臺北市,交通運輸,11,火車站,臺北市臺鐵松山車站,25.049244,121.578933
2,A,臺北市,交通運輸,11,火車站,臺北市臺鐵臺北車站,25.047379,121.516891
3,A,臺北市,交通運輸,11,火車站,松山火車站,25.049388,121.579563
4,A,臺北市,交通運輸,11,火車站,南港火車站,25.053683,121.606943


In [4]:
facility_data.shape

(58323, 8)

## 篩選重要設施
- 交通：火車站、客運站、捷運站、高鐵站
- 教育：國小、國中、幼稚園、托兒所
- 公園：一般公園、里鄰公園
- 娛樂：娛樂設施、商店、超級市場、大型購物中心、百貨公司
- 其他：醫療保健、金融機構、加油站
- 嫌惡設施：產業用水及設施、工廠、主要道路建設

In [5]:
facility_data['小類別名稱'].unique()

array(['火車站', '客運站', '捷運站', '飯店', '碼頭', '高鐵站', '航空站', '主要道路建設', '國小',
       '國中', '高中', '專科', '大學及研究所', '幼稚園、托兒所', '傳統市場', '生活百貨量販', '一般公園',
       '里鄰公園', '觀光遊憩設施', '停車場地', '服務性設施', '行政機關', '醫療保健', '加油站', '百貨公司',
       '金融機構', '娛樂設施', '電力資源', '產業用水及設施', '工廠', '商店', '超級市場', '大型購物中心'],
      dtype=object)

In [5]:
data = {}
for i in facility_data['小類別名稱'].unique():
    data['最近'+i] = []
df = pd.DataFrame(data)
df

,最近火車站,最近客運站,最近捷運站,最近飯店,最近碼頭,最近高鐵站,最近航空站,最近主要道路建設,最近國小,最近國中,...,最近加油站,最近百貨公司,最近金融機構,最近娛樂設施,最近電力資源,最近產業用水及設施,最近工廠,最近商店,最近超級市場,最近大型購物中心


In [6]:
def train_coordination(x):
  x1, y1 = x['橫坐標'] ,x['縱坐標']
  proj = pyproj.Transformer.from_crs(3826, 4326, always_xy=True)
  x2, y2 = proj.transform(x1, y1)
  return x2, y2

train[['lon','lat']] = train.apply(train_coordination,axis=1,result_type='expand')
new_train = train.drop(['橫坐標','縱坐標'], axis=1)

In [7]:
# 轉換 train 和 facility_data 為 GeoDataFrame
train['geometry'] = [Point(xy) for xy in zip(train['lon'], train['lat'])]
train = gpd.GeoDataFrame(train, geometry='geometry')

# 設定crs座標 TWD97
facility_data = gpd.GeoDataFrame(facility_data, geometry=gpd.points_from_xy(facility_data['經度'], facility_data['緯度']))
facility_data.crs = "EPSG:3826"
new_train.crs = "EPSG:3826"

In [9]:
for i in facility_data['小類別名稱'].unique():
    # 選取特定設施，計算該設施對房子的最小距離
    specific_facility = facility_data[facility_data['小類別名稱']==i]

In [8]:
# 計算附近數量的函數
def min_nearby_facility_distance(point, facility_data, max_distance=500):
    
    full_distance = (facility_data.geometry.distance(point) * 101.77545 * 1000).sort_values().reset_index(drop=True)
    # print(full_distance[0])
    
    return full_distance[0]

for i in facility_data['小類別名稱'].unique():
    
    # 選取特定設施，計算該設施對房子的最小距離
    specific_facility = facility_data[facility_data['小類別名稱']==i]
    train['最近'+i] = train.geometry.apply(min_nearby_facility_distance, facility_data = specific_facility)
    # min_nearby_facility_distance(train.geometry[0], facility_data)

KeyboardInterrupt: 

In [14]:
train.to_csv("./最近設施距離.csv", index=False)

In [34]:
facility_data['縣市'].unique()

array(['臺北市', '臺中市', '臺南市', '基隆市', '高雄市', '新北市', '宜蘭縣', '桃園縣', '苗栗縣',
       '嘉義市', '新竹縣', '南投縣', '彰化縣', '雲林縣', '新竹市', '嘉義縣', '屏東縣', '花蓮縣',
       '臺東縣', '金門縣', '澎湖縣', '連江縣'], dtype=object)

In [32]:
facility_data['小類別名稱'].unique()

array(['火車站', '客運站', '捷運站', '飯店', '碼頭', '高鐵站', '航空站', '主要道路建設', '國小',
       '國中', '高中', '專科', '大學及研究所', '幼稚園、托兒所', '傳統市場', '生活百貨量販', '一般公園',
       '里鄰公園', '觀光遊憩設施', '停車場地', '服務性設施', '行政機關', '醫療保健', '加油站', '百貨公司',
       '金融機構', '娛樂設施', '電力資源', '產業用水及設施', '工廠', '商店', '超級市場', '大型購物中心'],
      dtype=object)

In [45]:
for i in facility_data['縣市'].unique():
    specific_data = facility_data[facility_data['縣市'] == i]
    print(len(specific_data['小類別名稱'].unique()))

27
27
29
22
31
26
25
24
24
25
23
23
23
27
23
27
29
25
23
20
22
17


In [30]:
a = facility_data[facility_data['縣市'] == '臺北市'] #facility_data['小類別名稱'] == '商店' & 
a[a['小類別名稱'] == '商店']

,縣市代號,縣市,大類別,小類別代號,小類別名稱,地標名稱,緯度,經度,geometry
